💻 Task 1: Batch Processing Simulation (Python)

In [1]:
import subprocess

scripts = ['script1.py', 'script2.py', 'script3.py']

print(" Batch Processing Started  ")

for script in scripts:
    print(f"\nExecuting {script} ...")
    # Tries to execute the script using the python3 interpreter
    subprocess.call(['python3', script]) 

print("\n Batch Processing Completed --")

 Batch Processing Started  

Executing script1.py ...

Executing script2.py ...

Executing script3.py ...

 Batch Processing Completed --


⏰ Task 2: System Startup and Logging (Python)

In [2]:
import multiprocessing
import logging
import time

# Configuration for logging: specifies file, level, and format
logging.basicConfig(
    filename='system_log.txt',
    level=logging.INFO,
    format='%(asctime)s - %(processName)s - %(message)s'
)

def process_task(name):
    # Log process start
    logging.info(f"{name} started")
    time.sleep(2) # Simulate work being done
    # Log process termination
    logging.info(f"{name} terminated")

if __name__ == '__main__':
    print("System Booting...\n")
    
    # Create two processes
    p1 = multiprocessing.Process(target=process_task, args=("Process-1",))
    p2 = multiprocessing.Process(target=process_task, args=("Process-2",))
    
    # Start the processes
    p1.start()
    p2.start()
    
    # Wait for both processes to complete
    p1.join()
    p2.join()
    
    print("\nSystem Shutdown.")

System Booting...


System Shutdown.


🛠️ Task 3: System Calls and IPC (Python - fork, exec, pipe)

In [4]:
from multiprocessing import Process, Pipe

def child_process(conn):
    # Receive a message from the parent
    msg = conn.recv()
    print("Child received:", msg)

if __name__ == "__main__":
    # Create a pipe with two ends
    parent_conn, child_conn = Pipe()

    # Create the child process
    p = Process(target=child_process, args=(child_conn,))
    p.start()

    # Parent sends data through the pipe
    parent_conn.send("Hello from parent process via pipe!")

    # Wait for child to finish
    p.join()


🖥️ Task 4: VM Detection and Shell Interaction

In [9]:
import os
import platform
import subprocess

def detect_virtual_machine():
    print(" VM Detection")

    # Method 1: Check system manufacturer using dmidecode (Linux only)
    try:
        output = subprocess.check_output("dmidecode -s system-manufacturer", shell=True).decode().lower()
        if "vmware" in output or "virtualbox" in output or "qemu" in output:
            print("Running inside a Virtual Machine.")
            return
    except:
        pass # Ignore errors if dmidecode is not found or fails

    # Method 2: Check CPU virtualization flag in /proc/cpuinfo (Linux only)
    try:
        cpuinfo = ""
        with open("/proc/cpuinfo") as f:
            cpuinfo = f.read().lower()
        
        if "hypervisor" in cpuinfo:
            print("Hypervisor flag detected VM environment.")
            return
    except:
        pass # Ignore errors if /proc/cpuinfo is not accessible or fails

    # Method 3: Check platform metadata
    if "virtual" in platform.platform().lower():
        print("VM pattern found in platform data.")
        return

    print("System appears to be running on a physical machine.")

detect_virtual_machine()

 VM Detection
System appears to be running on a physical machine.


⚙️ Task 5: CPU Scheduling Algorithms (Python)

In [8]:
# FCFS (First-Come, First-Served)
def fcfs(burst_times):
    wt = [0] * len(burst_times)
    tat = [0] * len(burst_times)
    
    # Calculate Waiting Time (WT)
    for i in range(1, len(burst_times)):
        # WT[i] is the WT of the previous process + its Burst Time (BT)
        wt[i] = wt[i-1] + burst_times[i-1] 
    
    # Calculate Turnaround Time (TAT)
    for i in range(len(burst_times)):
        # TAT = WT + BT
        tat[i] = wt[i] + burst_times[i]
    
    return wt, tat

# SJF (Shortest Job First)
def sjf(burst_times):
    # Pair process index (pid) with its burst time (bt)
    indexed = list(enumerate(burst_times)) 
    # Sort processes by burst time (shortest job first)
    indexed.sort(key=lambda x: x[1]) 
    
    wt = [0] * len(burst_times)
    tat = [0] * len(burst_times)
    current_time = 0
    
    for pid, bt in indexed:
        wt[pid] = current_time
        tat[pid] = current_time + bt
        current_time += bt # Update current time by adding the process's BT
        
    return wt, tat

# Priority Scheduling
def priority_scheduling(burst_times, priority):
    # Pair process index (pid) with a tuple of (burst_time, priority)
    indexed = list(enumerate(zip(burst_times, priority))) 
    # Sort processes by priority (assuming lower number means higher priority)
    indexed.sort(key=lambda x: x[1][1]) 
    
    wt = [0] * len(burst_times)
    tat = [0] * len(burst_times)
    current_time = 0
    
    for pid, (bt, pr) in indexed:
        wt[pid] = current_time
        tat[pid] = current_time + bt
        current_time += bt
        
    return wt, tat

# Round Robin
def round_robin(burst_times, quantum):
    remaining_burst = burst_times[:] # Create a copy of burst times
    n = len(burst_times)
    current_time = 0
    wt = [0] * n
    tat = [0] * n
    
    done = False
    while not done:
        done = True
        
        for i in range(n):
            if remaining_burst[i] > 0:
                done = False # A process is still running
                
                if remaining_burst[i] > quantum:
                    # Process runs for the time quantum
                    current_time += quantum
                    remaining_burst[i] -= quantum
                else:
                    # Process finishes execution
                    current_time += remaining_burst[i]
                    # Calculate Waiting Time: finish time - original burst time
                    wt[i] = current_time - burst_times[i] 
                    remaining_burst[i] = 0
    
    # Calculate Turnaround Time
    for i in range(n):
        tat[i] = wt[i] + burst_times[i]
        
    return wt, tat

# MAIN PROGRAM
if __name__ == "__main__":
    print("\nCPU Scheduling Algorithms\n")
    
    n = int(input("Enter number of processes: "))
    
    burst = []
    for i in range(n):
        burst.append(int(input(f"Enter burst time for P{i+1}: ")))
        
    # FCFS
    print("\nFCFS Scheduling:")
    wt_fcfs, tat_fcfs = fcfs(burst)
    print("WT =", wt_fcfs)
    print("TAT =", tat_fcfs)
    
    # SJF
    print("\nSJF Scheduling:")
    wt_sjf, tat_sjf = sjf(burst)
    print("WT =", wt_sjf)
    print("TAT =", tat_sjf)
    
    # Priority Scheduling
    print("\nPriority Scheduling:")
    priority_list = []
    for i in range(n):
        priority_list.append(int(input(f"Enter priority for P{i+1}: ")))
        
    wt_priority, tat_priority = priority_scheduling(burst, priority_list)
    print("WT =", wt_priority)
    print("TAT =", tat_priority)
    
    # Round Robin
    print("\nRound Robin Scheduling:")
    quantum = int(input("Enter time quantum: "))
    
    wt_rr, tat_rr = round_robin(burst, quantum)
    print("WT =", wt_rr)
    print("TAT =", tat_rr)


CPU Scheduling Algorithms


FCFS Scheduling:
WT = [0, 5, 8, 16]
TAT = [5, 8, 16, 22]

SJF Scheduling:
WT = [3, 0, 14, 8]
TAT = [8, 3, 22, 14]

Priority Scheduling:
WT = [3, 0, 8, 16]
TAT = [8, 3, 16, 22]

Round Robin Scheduling:
WT = [9, 3, 14, 14]
TAT = [14, 6, 22, 20]
